## Embedding the data

**Using codeBERT for the classification task**

Importing librairies

In [1]:
import torch
import warnings
import pandas as pd
import numpy as np
import pickle
import os
from pathlib import Path
from transformers import AutoTokenizer, AutoModel, pipeline
from scipy.spatial.distance import cosine
from transformers import PLBartForConditionalGeneration, PLBartTokenizer
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline

In [2]:
# ignore warnings
warnings.filterwarnings('ignore')

Load cobeBERT base model and tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

In [3]:
DATA_PATH = "../data/"

### 1. TESTS

Basic test

In [5]:
nl_tokens = tokenizer.tokenize("returns maximum value")
code_tokens = tokenizer.tokenize("def max(a,b): if a>b: return a else return b")

tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]

tokens_ids = tokenizer.convert_tokens_to_ids(tokens)

context_embeddings = model(torch.tensor(tokens_ids)[None,:])[0]
context_embeddings

tensor([[[-0.1342,  0.3599,  0.0361,  ..., -0.2329, -0.3161,  0.3294],
         [-0.7013,  0.1173,  0.0651,  ..., -0.3564, -0.2514,  0.2654],
         [-0.3371,  0.1115,  0.4299,  ..., -0.2361, -0.1156,  0.8037],
         ...,
         [-0.4057,  0.1638,  0.4813,  ..., -0.1657, -0.2869,  0.7310],
         [-0.3968,  0.4617,  0.5130,  ..., -0.3096, -0.6014,  0.4400],
         [-0.1354,  0.3618,  0.0367,  ..., -0.2342, -0.3183,  0.3317]]],
       grad_fn=<NativeLayerNormBackward0>)

Test codeBERT for md only

In [4]:
nl_tokens = tokenizer.tokenize("returns category of a notebook")

tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]

tokens_ids = tokenizer.convert_tokens_to_ids(tokens)

context_embeddings_md = model(torch.tensor(tokens_ids)[None,:])[0]
context_embeddings_md

tensor([[[-0.1342,  0.3342,  0.0396,  ..., -0.2193, -0.3251,  0.3286],
         [-0.2917,  0.4377,  0.1195,  ..., -0.3086, -0.5924,  0.1050],
         [ 0.0023,  0.2300, -0.0107,  ...,  0.0430, -0.4059,  0.0055],
         [-0.1335,  0.3338,  0.0403,  ..., -0.2191, -0.3248,  0.3279]]],
       grad_fn=<NativeLayerNormBackward0>)

Test codeBERT for code only

In [38]:
code_tokens = tokenizer.tokenize(
    """for i in data.index:
    if data.loc[i]['subcategory'] == sub:
        categ = data.loc[i]['category']
    """)

tokens=[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]

tokens_ids = tokenizer.convert_tokens_to_ids(tokens)

context_embeddings_code = model(torch.tensor(tokens_ids)[None,:])[0]
context_embeddings_code

tensor([[[-0.1264,  0.3699,  0.0476,  ..., -0.1938, -0.2802,  0.3096],
         [-0.5152,  0.4615,  0.2745,  ..., -0.4479, -0.3483,  0.1919],
         [-0.1253,  0.3687,  0.0483,  ..., -0.1938, -0.2787,  0.3072]]],
       grad_fn=<NativeLayerNormBackward0>)

In [15]:
context_embeddings.shape

torch.Size([1, 24, 768])

In [12]:
type(context_embeddings)

torch.Tensor

In [14]:
context_embeddings.shape
# 1 = number of batches
# 23 = number of tokens
# 768 = number of hidden units

torch.Size([1, 23, 768])

##### Testing embedding aggregation using averaging

In [40]:
# convert tensor into np array
tensor_np = context_embeddings_md.cpu().detach().numpy()
# average of embeddings of the tokens in the sequence
avg_md = np.mean(tensor_np[0], axis=0)
len(avg_md)

768

In [41]:
# convert tensor into np array
tensor_np = context_embeddings_code.cpu().detach().numpy()
# average of embeddings of the tokens in the sequence
avg_code = np.mean(tensor_np[0], axis=0)
len(avg_code)

768

Check whether the code and md sequences are semantically similar using cosine similarity

In [42]:
cos = 1 - cosine(avg_md, avg_code)
cos

0.9352593421936035

##### Test CodeBERT for mask prediction

In [14]:
# load model and tokenizer
mlm_model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm")
mlm_tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")

In [26]:
CODE = "<mask> matplotlib.pyplot as plt"
mlm_model_name = "microsoft/codebert-base-mlm"
fill_mask = pipeline('fill-mask', model=mlm_model, tokenizer=mlm_tokenizer)

outputs = fill_mask(CODE)
print(outputs)

[{'score': 0.7230142951011658, 'token': 6595, 'token_str': ' import', 'sequence': ' importmatplotlib.pyplot as plt'}, {'score': 0.1273956149816513, 'token': 4, 'token_str': '.', 'sequence': '.matplotlib.pyplot as plt'}, {'score': 0.03073594532907009, 'token': 1215, 'token_str': '_', 'sequence': '_matplotlib.pyplot as plt'}, {'score': 0.023831013590097427, 'token': 479, 'token_str': '.', 'sequence': '.matplotlib.pyplot as plt'}, {'score': 0.010915211401879787, 'token': 18134, 'token_str': ' _', 'sequence': ' _matplotlib.pyplot as plt'}]


##### Test PLBART for code-to-text & text-to-code translation

In [26]:
plbarttokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-python-en_XX", src_lang="python", tgt_lang="en_XX")
plbart_model = PLBartForConditionalGeneration.from_pretrained("uclanlp/plbart-python-en_XX")

In [33]:
example_python_phrase = """def average_embeddings(embedding_tensor):

    avg_rep = np.empty(1)
    if embedding_tensor is not None:
        if type(embedding_tensor) == torch.Tensor:
            # convert tensor into np array
            tensor_np = embedding_tensor.cpu().detach().numpy()
            # average of embeddings of the tokens in the sequence
            avg_rep = np.mean(tensor_np[0], axis=0)
        else: 
            tensor_np = embedding_tensor
            # average of embeddings of the tokens in the sequence
            avg_rep = np.mean(tensor_np, axis=0)
        # return average representation of a sequence of tokens
        return avg_rep
"""
inputs = plbarttokenizer(example_python_phrase, return_tensors="pt")
translated_tokens = plbart_model.generate(**inputs, decoder_start_token_id=plbarttokenizer.lang_code_to_id["en_XX"])
plbarttokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

'average representation of embeddings in a sequence of tokens'

*Note to self: PLBART gives pretty good results!*

##### Train and test T5 for text to text tranformation


Use this https://huggingface.co/docs/transformers/model_doc/t5

### 2. EMBEDDING REAL DATA

Load dataset containing clean text (code and md cells for each notebook and their category)

In [34]:
with open(DATA_PATH + 'text_dataset.pkl', 'rb') as f:
    text_data = pickle.load(f)

In [35]:
text_data.head()

,cell-type,source,title,tag
344966,code,"POI_data = gpd.read_file(""../input/geospatial...",your-first-map.ipynb,reinforcement learning
344967,markdown,next we create a map from all four geodatafram...,your-first-map.ipynb,reinforcement learning
344968,code,"ax = counties.plot(figsize=(10,10), color='no...",your-first-map.ipynb,reinforcement learning
344969,markdown,it looks like the northeastern part of the sta...,your-first-map.ipynb,reinforcement learning
344970,markdown,have questions or comments visit the course di...,your-first-map.ipynb,reinforcement learning


In [8]:
text_data.shape

(332354, 4)

Embedding each cell's tokens and averaging them to get a representation for the row

In [5]:
def embed_single_row(text, row):

    '''
    gets a text string (code or md) and the row index
    tests for sequences longer than 510 not to exceed the max_len limit of BERT model
    tokenizes and returns embeddings tensor
    
    '''
    
    nl_tokens = tokenizer.tokenize(text)

    # we choose max_len to be 510 as the tokenizer then adds 2 special tokens <s> and </s>
    if len(nl_tokens) > 510: 
        nl_tokens = nl_tokens[:510]

    tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]

    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    row_embeddings = model(torch.tensor(tokens_ids)[None,:])[0]

    if row_embeddings is not None:
        return row_embeddings


In [6]:
def average_embeddings(embedding_tensor):

    '''
    gets the returned embeddings tensor of a sequence and averages it 
    to return one 768 dim vector for each row in the dataframe

    '''

    avg_rep = np.empty(1)
    if embedding_tensor is not None:
        if type(embedding_tensor) == torch.Tensor:
            # convert tensor into np array
            tensor_np = embedding_tensor.cpu().detach().numpy()
            # average of embeddings of the tokens in the sequence
            avg_rep = np.mean(tensor_np[0], axis=0)
        else: 
            tensor_np = embedding_tensor
            # average of embeddings of the tokens in the sequence
            avg_rep = np.mean(tensor_np, axis=0)
        # return average representation of a sequence of tokens
        return avg_rep

We proceed by chunks of the dataset for time optimization reasons

In [ ]:
row_avg_embedding = []
for row in text_data.index[:166177]:
    embedding = embed_single_row(text_data.loc[row, 'source'], row)
    row_avg_embedding.append((average_embeddings(embedding),  text_data.loc[row, 'title'], text_data.loc[row, 'tag']))
    print(row)
    
# build intermediary dataframe of averaged row vectors, titles and tags 
interm_df01 = pd.DataFrame(row_avg_embedding, columns = ['row_vector', 'title', 'tag'])

In [ ]:
row_avg_embedding = []
for row in text_data.index[166177:(166177+100000)]:
    embedding = embed_single_row(text_data.loc[row, 'source'], row)
    row_avg_embedding.append((average_embeddings(embedding),  text_data.loc[row, 'title'], text_data.loc[row, 'tag']))
    print(row)
# build intermediary dataframe of averaged row vectors, titles and tags 
interm_df02 = pd.DataFrame(row_avg_embedding, columns = ['row_vector', 'title', 'tag'])

In [ ]:
row_avg_embedding = []
for row in text_data.index[266177:]:
    embedding = embed_single_row(text_data.loc[row, 'source'], row)
    row_avg_embedding.append((average_embeddings(embedding),  text_data.loc[row, 'title'], text_data.loc[row, 'tag']))
    print(row)
# build intermediary dataframe of averaged row vectors, titles and tags 
interm_df03 = pd.DataFrame(row_avg_embedding, columns = ['row_vector', 'title', 'tag'])

Save the vectorized chunks to pkl files

In [49]:
interm_df01.to_pickle(DATA_PATH + 'interm_df_0_166176.pkl')
interm_df02.to_pickle(DATA_PATH + 'interm_df_166177_266177.pkl')
interm_df03.to_pickle(DATA_PATH + 'interm_df_266177_332354.pkl')

Now, let's construct our final dataset by grouping by notebook titles and averaging the vectors

In [21]:
pickles_dir = Path(DATA_PATH + 'pickles\\').glob('*')
vect_data = pd.DataFrame()

vect_data = pd.concat([pd.read_pickle(DATA_PATH + 'pickles\\' + filename.name) for filename in pickles_dir])
vect_data.head()

,row_vector,title,tag
0,"[-0.15601629, 0.07173749, 0.52512264, 0.035489...",0-9-try-better-parameters-better-score.ipynb,regression
1,"[-0.003272749, 0.27969736, 0.29870665, 0.12947...",0-9-try-better-parameters-better-score.ipynb,regression
2,"[-0.14450021, 0.05994332, -0.022545334, -0.018...",0-9-try-better-parameters-better-score.ipynb,regression
3,"[0.025118893, 0.24624567, 0.13547924, 0.313845...",0-9-try-better-parameters-better-score.ipynb,regression
4,"[-0.48143202, 0.40252736, 0.4532451, -0.054363...",0-9-try-better-parameters-better-score.ipynb,regression


In [22]:
vect_data.reset_index(drop=True, inplace=True)

In [26]:
vect_data.index.duplicated().sum()

0

In [23]:
vect_data.shape

(332354, 3)

In [28]:
vect_data.isna().sum()

row_vector    0
title         0
tag           0
dtype: int64

In [34]:
def group_and_avg(df):
    grp_avg_embedding = []

    grp_by_title = df.groupby('title').groups
    grp_titles = list(grp_by_title.keys())
    for grp_title in grp_titles:
        # get indices range for each title
        grp_indices = grp_by_title[grp_title]
        # get row embeddings for each notebook 
        grp_vecs = df.loc[grp_indices, 'row_vector']
        # get group tag
        grp_tag = df.loc[grp_indices[0], 'tag']
        # call average method
        grp_avg_embedding.append((list(average_embeddings(grp_vecs)), grp_tag)) 
        
    vect_data = pd.DataFrame(grp_avg_embedding, columns = ['notebook_vector', 'tag'])
    return vect_data

In [35]:
vect_data_final = group_and_avg(vect_data)
vect_data_final.head()

,notebook_vector,tag
0,"[-0.34790626, 0.29865852, 0.30619365, 0.055174...",computer vision
1,"[-0.3819649, 0.31716973, 0.33679396, 0.0810977...",clustering
2,"[-0.33451593, 0.2831545, 0.29143128, 0.0433948...",computer vision
3,"[-0.27561176, 0.26736438, 0.2665188, 0.0765375...",nlp
4,"[-0.24364452, 0.28803557, 0.2663721, 0.1090390...",classification


In [36]:
vect_data_final.shape

(6260, 2)

In [38]:
vect_data_final.to_csv(DATA_PATH + 'vect_data_final.csv')